In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tf.__version__ 

'2.9.2'

Part 1 - Data Preprocessing

Generating images for the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   )

**Generating images for the Test set**

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

**Creating the Training set**

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/asl_dataset_3/asl-dataset/train',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')


Found 31342 images belonging to 26 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/asl_dataset_3/asl-dataset/val',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 8493 images belonging to 26 classes.


**Part 2 - Building and**
**Initialization of CNN**

In [ ]:
classifier = tf.keras.models.Sequential()

**Step 1 - Convolution**

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

**Step 2 - Pooling**

In [ ]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

**Adding a second convolutional layer**

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

**Step 3 - Flattening**

In [ ]:
classifier.add(tf.keras.layers.Flatten())

**Step 4 - Full Connection**

In [ ]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))

classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))

classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))

classifier.add(tf.keras.layers.Dense(units=26, activation='softmax')) # softmax for more than 2

**Part 3 - Training and
Compiling the CNN**

In [ ]:

classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

**Training the CNN on the Training set and evaluating it on the Test set**

In [ ]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [ ]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = test_set)

Epoch 1/5
3135/3135 [==============================] - 8614s 3s/step - loss: 2.2326 - accuracy: 0.3213 - val_loss: 0.5956 - val_accuracy: 0.8259
Epoch 2/5
3135/3135 [==============================] - 805s 257ms/step - loss: 0.9864 - accuracy: 0.6841 - val_loss: 0.2658 - val_accuracy: 0.9285
Epoch 3/5
3135/3135 [==============================] - 789s 252ms/step - loss: 0.7014 - accuracy: 0.7811 - val_loss: 0.1795 - val_accuracy: 0.9490
Epoch 4/5
3135/3135 [==============================] - 776s 248ms/step - loss: 0.5780 - accuracy: 0.8228 - val_loss: 0.1274 - val_accuracy: 0.9637
Epoch 5/5
3135/3135 [==============================] - 767s 245ms/step - loss: 0.4963 - accuracy: 0.8491 - val_loss: 0.1286 - val_accuracy: 0.9627


**Saving the Model**

In [ ]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
